### Data Description

train.csv: A full training dataset with the following attributes:

id: unique id for a news article \
title: the title of a news article \
author: author of the news article \
text: the text of the article; could be incomplete \
label: a label that marks the article as potentially unreliable \
    1: unreliable \
    0: reliable 


### Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from typing import Type

In [ ]:
import nltk
nltk.download('stopwords')
# print(stopwords.words(fileids='english'))

### Data pre-processing

#### Loading the dataset

In [ ]:
news_dataset =  pd.read_csv('train.csv')
news_dataset.shape

#### Data analysis

In [ ]:
news_dataset.head()

Either we can impute the dataset or we can remove the rows with null values

### In first approach, we use title and author to check for fake news and just impute with empty string

In [ ]:
news_dataset = news_dataset.fillna('')
news_dataset.isna().sum()

merging the author name and news title

In [ ]:
news_dataset['content'] = news_dataset['author'] + ' ' +  news_dataset['title']

In [ ]:
news_dataset.head()

#### Data pre-processing

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content: str) -> str:
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words(fileids='english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
news_dataset.head()

#### Splitting Dataset

In [ ]:
X = news_dataset['content'].values
y = news_dataset['label'].values

### Converting texual data to numerical data

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

### Train, Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

### Training the classification model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)

### Evaluation

#### accuracy score

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(y_train, X_train_prediction)

In [ ]:
training_data_accuracy

In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(y_test, X_test_prediction)

In [ ]:
test_data_accuracy

### Predictive System

In [ ]:
X_new = X_test[0]
prediction = model.predict(X_new)

if(prediction == 0):
    print("The news is real")
else:
    print("The news is fake")